In [1]:
!pip install transformers
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 829.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import TextStreamer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
# from huggingface_hub import login

# # API 키 입력 받기
# login()

In [4]:
from unsloth import FastLanguageModel
import torch


# 설정
max_seq_length=1024  # 최대 시퀀스 길이
dtype = None  # 자동 감지를 위해 None 사용
load_in_4bit = True  # 4bit 양자화 활성화

fourbit_models = [
    "unsloth/gemma-2-2b",
] 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-2b",
    max_seq_length=max_seq_length, 
    dtype=dtype,
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부
)


==((====))==  Unsloth 2024.9.post4: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9.post4 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [6]:
import pandas as pd
from datasets import Dataset

# 데이터 로드
df = pd.read_csv('/kaggle/input/text-counseling/text.csv')
indexes_to_drop = [1530, 26003, 26004,46349,48630, 56335,67122,67123,67124,67125,67126,67127, 77414, 77415, 
                   82947, 82949, 90078, 90079, 121210, 121211, 192316, 192317,323528,314734, 314735, 348065]
                                      

# 인덱스를 기준으로 행 삭제
df = df.drop(indexes_to_drop)

# 인덱스 리셋 (기존 인덱스는 삭제하고 기본 숫자 인덱스로 변경)
df.reset_index(drop=True, inplace=True)

df.loc[44570, 'type'] = 'Q'
df.loc[44572, 'type'] = 'Q'
df.loc[82933, 'type'] = 'A'
df.loc[289415, 'type'] = 'A'


# 질문과 답변 쌍 만들기
df_pairs = []
for i in range(0, len(df), 2):  # 2씩 증가하며 쌍을 만듭니다.
    if df.iloc[i]['type'] == 'Q':
        question = df.iloc[i]['text']
        answer = df.iloc[i + 1]['text'] if i + 1 < len(df) and df.iloc[i + 1]['type'] == 'A' else ''
        df_pairs.append({'instruction': question, 'input': '', 'output': answer})

# DataFrame으로 변환
df2 = pd.DataFrame(df_pairs)


# # 데이터 자르기
# df2 = df2.head(100000)

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df2)

# EOS_TOKEN 설정
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt 포맷팅 함수 수정
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# formatting_prompts_func 수정
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # EOS_TOKEN을 추가합니다.
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text": texts }

# 데이터셋에 formatting_prompts_func 함수를 적용합니다.
dataset = dataset.map(formatting_prompts_func, batched=True)

# 포맷된 데이터셋 확인
print(dataset)


Map:   0%|          | 0/180395 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 180395
})


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 60,
        num_train_epochs=3,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 2,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="none",
    ),
)



Map (num_proc=2):   0%|          | 0/180395 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 180,395 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 2
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
2,3.263400
4,3.500300
6,3.148200
8,3.137400
10,3.035900
12,2.874600
14,2.777000
16,2.597400
18,2.419900
20,2.385500


In [9]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

In [10]:


# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "요새 기분이 어떠니?",  # 상담사 질문
            "", # input
            "",  # output
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=20,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
요새 기분이 어떠니?

### Input:


### Response:
좋아요.<eos>


In [11]:

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "뭐하고 싶어?",  # 상담사 질문
            "", # input
            "",  # output
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=50,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
뭐하고 싶어?

### Input:


### Response:
아니요.<eos>


In [12]:

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "왜 그랬니?",  # 상담사 질문
            "", # input
            "",  # output
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
왜 그랬니?

### Input:


### Response:
그냥 잠깐 쉬어야 했어요.<eos>


In [13]:

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "너는 누구랑 살아?",  # 상담사 질문
            "", # input
            "",  # output
        )
    ],
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

<bos>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
너는 누구랑 살아?

### Input:


### Response:
아니요.<eos>


In [14]:
output_dir = './counseling_trainer_model'
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('./counseling_trainer_model/tokenizer_config.json',
 './counseling_trainer_model/special_tokens_map.json',
 './counseling_trainer_model/tokenizer.model',
 './counseling_trainer_model/added_tokens.json',
 './counseling_trainer_model/tokenizer.json')